# Modelos Predictivos: Rentas Cedidas Municipales (Multi-Horizonte)
# SARIMAX, Prophet, XGBoost y Deep Learning

---

## 📋 Proyecto de Grado
**Título**: Predicción del Comportamiento de las Rentas Cedidas en el Financiamiento del Régimen Subsidiado de Salud a Nivel Municipal mediante Modelos de Machine Learning

**Objetivo Mejorado**: Desarrollar modelos predictivos multi-horizonte (Mensual, Bimestral, Trimestral) para estimar ingresos por Rentas Cedidas, integrando mejores prácticas de limpieza y feature engineering sugeridas por **NotebookLM**.

### 🧠 NotebookLM Insights Integrados:
1. **Interpolación Temporal**: Manejo de valores faltantes preservando tendencias.
2. **Codificación Cíclica**: Transformación Seno/Coseno para variables temporales.
3. **Análisis Multi-Escala**: Evaluación en diferentes agregaciones temporales para robustez.

---

**Fecha**: Octubre-Diciembre 2025 (Periodo de Evaluación)

## 1. Configuración del Entorno

In [ ]:
# Importaciones generales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import sys
import os
from pathlib import Path
from datetime import datetime

# Agregar directorio de scripts al path para importar módulos locales
current_dir = Path(os.getcwd())
scripts_dir = current_dir.parent / 'scripts'
sys.path.append(str(scripts_dir))

try:
    import config
    import utils
    print("✅ Módulos locales (config, utils) importados correctamente.")
except ImportError as e:
    print(f"⚠️ Advertencia: No se pudieron importar módulos locales: {e}")

# Modelos Econométricos
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Modelos ML y Prophet
from prophet import Prophet
# from neuralprophet import NeuralProphet # No instalado en este entorno
import xgboost as xgb

# Deep Learning
import torch
import torch.nn as nn

# Optimización y Validación
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✅ Entorno configurado.")

## 2. Ingeniería de Características Dinámica (NotebookLM Insights)

### 💡 NotebookLM Insight: Feature Engineering para Series Temporales
Según el análisis de **NotebookLM (Video 7)**, para modelos de Deep Learning y Machine Learning en series temporales es crucial:
1. **Preservar la Ciclidad**: Usar transformaciones Seno/Coseno para meses y trimestres evitar que el modelo interprete "Mes 12" y "Mes 1" como distantes.
2. **Ventanas de Tiempo (Lags)**: Incorporar rezagos (t-1, t-12) permite al modelo capturar autocorrelación directa.
3. **Diferenciación**: Estabilizar la media eliminando tendencias lineales simples.

In [ ]:
def feature_engineering_dynamic(df_input, freq='M'):
    """
    Genera features dinámicamente según la frecuencia (Horizonte) solicitada.
    """
    df = df_input.copy()
    
    # Asegurar datetime
    df['fecha'] = pd.to_datetime(df['fecha'])
    
    # Agregación por el horizonte deseado (Suma de Recaudo)
    df_agg = df.groupby(pd.Grouper(key='fecha', freq=freq))['recaudo'].sum().reset_index()
    df_agg = df_agg.sort_values('fecha')
    
    # Generar variables de tiempo
    df_agg['mes'] = df_agg['fecha'].dt.month
    df_agg['trimestre'] = df_agg['fecha'].dt.quarter
    
    # --- NotebookLM Technique: Cyclical Encoding ---
    df_agg['sin_mes'] = np.sin(2 * np.pi * df_agg['mes'] / 12)
    df_agg['cos_mes'] = np.cos(2 * np.pi * df_agg['mes'] / 12)
    
    # --- NotebookLM Technique: Lags ---
    # Ajustamos lags según la frecuencia
    lags = [1, 2, 3] # Lags inmediatos
    if freq == 'M':
        lags.append(12) # Estacionalidad anual para mensual
    elif freq == 'Q':
        lags.append(4) # Estacionalidad anual para trimestral
        
    for lag in lags:
        df_agg[f'recaudo_lag{lag}'] = df_agg['recaudo'].shift(lag)
        
    # Diferenciación
    df_agg['recaudo_diff'] = df_agg['recaudo'].diff().fillna(0)
    
    # Eliminar NaNs generados por lags iniciales
    df_final = df_agg.dropna()
    
    return df_final

print("✅ Función de Feature Engineering Dinámica definida.")

## 3. Definición de Modelos

In [ ]:
# --- SARIMAX ---
def entrenar_sarimax(train_series, order=(1,1,1), seasonal_order=(1,1,1,12)):
    try:
        model = SARIMAX(train_series, order=order, seasonal_order=seasonal_order,
                        enforce_stationarity=False, enforce_invertibility=False)
        return model.fit(disp=False)
    except: return None

# --- Prophet ---
def entrenar_prophet(train_df, features_exogenas):
    model = Prophet(yearly_seasonality=True, weekly_seasonality=False, seasonality_mode='multiplicative')
    for reg in features_exogenas: model.add_regressor(reg)
    return model

# --- XGBoost ---
def entrenar_xgboost(X_train, y_train):
    model = xgb.XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=6)
    return model

# --- LSTM ---
class LSTMNet(nn.Module):
    def __init__(self, input_dim, hidden_dim=50, output_dim=1):
        super(LSTMNet, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

print("✅ Modelos definidos.")

## 4. Ejecución Multi-Horizonte (Mensual, Bimestral, Trimestral)

In [ ]:
# Cargar datos crudos depurados (Excel)
print(f"📂 Cargando datos depurados desde: {config.CLEANED_DATA_FILE}")
df_raw = utils.load_data(config.CLEANED_DATA_FILE)

horizontes = {
    'Mensual': 'M',
    'Bimestral': '2ME', # Using 2ME (Month End) standard
    'Trimestral': 'Q'
}

resultados_globales = []

for nombre_horizonte, frecuencia in horizontes.items():
    print(f"\n" + "="*60)
    print(f"🕐 Procesando Horizonte: {nombre_horizonte.upper()} ({frecuencia})")
    print("="*60)
    
    # 1. Feature Engineering específico para la frecuencia
    try:
        df_h = feature_engineering_dynamic(df_raw, freq=frecuencia)
    except Exception as e:
        if frecuencia == '2ME': 
             print("⚠️ Fallback a freq='2M' param old pandas...")
             df_h = feature_engineering_dynamic(df_raw, freq='2M')
        else: raise e

    # 2. Split (Oct-Dec 2025 Test)
    split_date = pd.Timestamp(config.TRAIN_CUTOFF_DATE)
    train = df_h[df_h['fecha'] <= split_date]
    test = df_h[df_h['fecha'] > split_date]
    
    print(f"   🚂 Train: {len(train)} registros | 🧪 Test: {len(test)} registros")
    if test.empty: 
        print("   ⚠️ Skip: No hay datos para test.")
        continue

    # 3. Entrenamiento y Predicción
    # --- SARIMAX ---
    try:
        ts_train = train.set_index('fecha')['recaudo']
        # Ajuste dinámico de estacionalidad
        seasonal_periods = {'M': 12, '2ME': 6, '2M': 6, 'Q': 4}
        s = seasonal_periods.get(frecuencia, 12)
        
        model_s = entrenar_sarimax(ts_train, seasonal_order=(1,1,1,s))
        
        if model_s:
            pred_s = model_s.forecast(steps=len(test))
            mape_s = mean_absolute_percentage_error(test['recaudo'], pred_s)
            print(f"   📊 SARIMAX (s={s}) MAPE: {mape_s:.2%}")
            resultados_globales.append({'Horizonte': nombre_horizonte, 'Modelo': 'SARIMAX', 'MAPE': mape_s})
        else:
             print("   ⚠️ SARIMAX no convergió.")
    except Exception as e: print(f"   ❌ SARIMAX Error: {e}")

    # --- XGBoost ---
    try:
        features = [c for c in df_h.columns if c not in ['fecha', 'recaudo']]
        X_train = train[features].select_dtypes(include=[np.number])
        X_test = test[features].select_dtypes(include=[np.number])
        
        model_x = xgb.XGBRegressor()
        model_x.fit(X_train, train['recaudo'])
        pred_x = model_x.predict(X_test)
        mape_x = mean_absolute_percentage_error(test['recaudo'], pred_x)
        print(f"   🌳 XGBoost MAPE: {mape_x:.2%}")
        resultados_globales.append({'Horizonte': nombre_horizonte, 'Modelo': 'XGBoost', 'MAPE': mape_x})
    except Exception as e: print(f"   ❌ XGBoost Error: {e}")

print("\n✅ Ejecución Multi-Horizonte Completada.")

## 5. Resumen de Resultados

In [ ]:
df_res = pd.DataFrame(resultados_globales)
if not df_res.empty:
    print(df_res.sort_values(['Horizonte', 'MAPE']))
    # Guardar reporte
    # df_res.to_excel("reporte_multihorizonte.xlsx")